In [1]:
!nvidia-smi

Sat Sep  2 08:05:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   47C    P0    77W / 300W |  10439MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
# from train_kle_bse import RankT5GPE
from torch import nn
import torch
from datasets import Dataset, DatasetDict
import datasets

In [4]:
tokenized_wow = datasets.load_from_disk("wow_qg_test_data")

In [5]:
# tokenized_wow["valid"] = Dataset.from_dict(tokenized_wow["valid"][:-7])

In [6]:
tokenized_wow

Dataset({
    features: ['attention_mask', 'input_ids', 'pass_label'],
    num_rows: 25732
})

In [7]:
model_ckpt = "wow_rank_t5-base_sym_kld_tem:T1S1/final"
# model_ckpt = "wow_kle_t5_base"
tokenizer = T5Tokenizer.from_pretrained(model_ckpt)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [8]:
def dec_inp(df):
    out = {"decoder_input_ids": []}
    for i, _ in enumerate(df["pass_label"]):
        out["decoder_input_ids"].append(tokenizer("<pad>", add_special_tokens=False)["input_ids"])
    
    return out

In [9]:
# test = tokenized_wow["test"].map(dec_inp, batched=True)
# test = tokenized_wow["test"]
test = tokenized_wow.map(dec_inp, batched=True)

Map:   0%|          | 0/25732 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(test["input_ids"][34])

'question: Cinematography is a type of motion picture, captured electronically by means of an image-sensor, produces an electrical charge</s> title: Polarization (waves) passage: Polarization (also polarisation) is a property applying to transverse waves that specifies the geometrical orientation of the oscillations. In a transverse wave, the direction of the oscillation is transverse to the direction of motion of the wave, so the oscillations can have different directions perpendicular to the wave direction. A simple example of a polarized transverse wave is vibrations traveling along a taut string "(see image)"; for example, in a musical instrument like a guitar string. Depending on how the string is plucked, the vibrations can be in a vertical direction, horizontal direction, or at any angle perpendicular to the string.</s>'

In [11]:
def p2p(df):
    out = {"input_ids": [], "attention_mask": []}
    for i, inpid in enumerate(df["input_ids"]):
        ide = inpid.index(1)
        out["input_ids"].append(inpid[:ide] + inpid[ide+1:])
        att = df["attention_mask"][i]
        out["attention_mask"].append(att[:ide] + att[ide+1:])
    return out
test = test.map(p2p, batched=True)

Map:   0%|          | 0/25732 [00:00<?, ? examples/s]

In [12]:
tokenizer.decode(test["input_ids"][34])

'question: Cinematography is a type of motion picture, captured electronically by means of an image-sensor, produces an electrical charge title: Polarization (waves) passage: Polarization (also polarisation) is a property applying to transverse waves that specifies the geometrical orientation of the oscillations. In a transverse wave, the direction of the oscillation is transverse to the direction of motion of the wave, so the oscillations can have different directions perpendicular to the wave direction. A simple example of a polarized transverse wave is vibrations traveling along a taut string "(see image)"; for example, in a musical instrument like a guitar string. Depending on how the string is plucked, the vibrations can be in a vertical direction, horizontal direction, or at any angle perpendicular to the string.</s>'

In [13]:
class RankT5GPE(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
        config.rank_score_index = 32019
        config.n_pass = 7
        config.output_hidden_states = True
        super().__init__(config)
        self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = int(batch_size_n/self.n_pass)

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
#             print(decoder_input_ids.size())
# <pad> k1 k2 k3 
#       -1 -2 4 --> 1
#            rs --> 2

        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
        rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :]) # bn * dsl * 512 --> bn * 512 --> bn * v 
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass) # bn * v --> bn --> b * n


        if labels != None:
            logits = out.logits
            batch_size_n, sequence_length, vocab_size = logits.size()

            logits_flat = logits.view(batch_size_n * sequence_length, vocab_size)
            labels_flat = labels.view(-1)
            mask = (labels_flat != -100)
            arry = torch.arange(batch_size_n * sequence_length).to(logits_flat.device)

            selected_logits = logits_flat[arry[mask], labels_flat[mask]]
            output_logits = torch.full((batch_size_n * sequence_length,), 0, dtype=logits.dtype, device=logits.device)
            output_logits[mask] = selected_logits

            output_logits = output_logits.view(batch_size_n, -1).sum(-1)
            out.gpe_score = output_logits.view(int(batch_size_n/self.n_pass), self.n_pass)

        else:
            out.gpe_score = None


        if pass_label != None:
            pass_label = pass_label[::self.n_pass] # bn
            # 3 --> log_softmax(rank_score)[3]
            # Q -> [23, 34, 48, 32] pl -> [2] rs -> [-2, -3, 5, -1] -> [_, _, __, _]
            # Q -> [23, 48, 34, 32] pl -> [2] rs -> [-2, 5, -3, -1] -> [_, _, __, _]
            rank_score = out.rank_score # [-2, 5, -3, -1] 
            gen_score = out.gpe_score # [-3, 4, -2, -1]

            loss_fct1 = nn.CrossEntropyLoss()
            loss_fct2 = nn.CrossEntropyLoss()

            rank_loss = loss_fct1(rank_score, pass_label.view(-1))
            gen_loss = loss_fct2(gen_score, pass_label.view(-1))

            loss = rank_loss + gen_loss
            out.loss = loss

        return out

In [14]:
from transformers.modeling_outputs import Seq2SeqLMOutput
class RankT5(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
        config.rank_score_index = 32019
        config.n_pass = 7
        config.output_hidden_states = True
        super().__init__(config)
#         self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = int(batch_size_n/self.n_pass)
        labels = None

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        if decoder_input_ids == None and labels == None:
            decoder_input_ids = torch.zeros((batch_size_n, 1), dtype=int).to(input_ids.device)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
            print(decoder_input_ids.size())


        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
#         rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :])
        rank_score = out.logits[:, 0, :] 
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass)
        


#         if labels != None:
#             logits = out.logits
#             batch_size_n, sequence_length, vocab_size = logits.size()

#             logits_flat = logits.view(batch_size_n * sequence_length, vocab_size)
#             labels_flat = labels.view(-1)
#             mask = (labels_flat != -100)
#             arry = torch.arange(batch_size_n * sequence_length).to(logits_flat.device)

#             selected_logits = logits_flat[arry[mask], labels_flat[mask]]
#             output_logits = torch.full((batch_size_n * sequence_length,), 0, dtype=logits.dtype, device=logits.device)
#             output_logits[mask] = selected_logits

#             output_logits = output_logits.view(batch_size_n, -1).sum(-1)
#             out.gpe_score = output_logits.view(int(batch_size_n/self.n_pass), self.n_pass)

#         else:
#             out.gpe_score = None
        loss = None


        if pass_label != None:
            pass_label = pass_label[::self.n_pass]
            rank_score = out.rank_score
#             gen_score = out.gpe_score

            loss_fct1 = nn.CrossEntropyLoss()
#             loss_fct2 = nn.CrossEntropyLoss()

            rank_loss = loss_fct1(rank_score, pass_label.view(-1))
#             gen_loss = loss_fct2(gen_score, pass_label.view(-1))

#             loss = rank_loss + gen_loss
            
            loss = rank_loss

        ret =  Seq2SeqLMOutput(
            loss=loss,
            logits=out.logits,
            past_key_values=out.past_key_values,
            decoder_hidden_states=out.decoder_hidden_states,
            decoder_attentions=out.decoder_attentions,
            cross_attentions=out.cross_attentions,
            encoder_last_hidden_state=out.encoder_last_hidden_state,
            encoder_hidden_states=out.encoder_hidden_states,
            encoder_attentions=out.encoder_attentions,
        )
        ret.rank_score = out.rank_score
        return ret

In [15]:
from transformers.modeling_outputs import Seq2SeqLMOutput
class RankT5GPE(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
#         config.rank_score_index = 32019
#         config.n_pass = 7
#         config.output_hidden_states = True
        super().__init__(config)
        self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = int(batch_size_n/self.n_pass)

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
#             print(decoder_input_ids.size())


        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
        rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :])
#         try:
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass)
#         except RuntimeError:
#             print(rank_score.size())
#             assert 2 == 3
            

        loss = None

        if labels != None:
            logits = out.logits
            batch_size_n, sequence_length, vocab_size = logits.size()

            logits_flat = logits.view(batch_size_n * sequence_length, vocab_size)
            labels_flat = labels.view(-1)
            mask = (labels_flat != -100)
            arry = torch.arange(batch_size_n * sequence_length).to(logits_flat.device)

            selected_logits = logits_flat[arry[mask], labels_flat[mask]]
            output_logits = torch.full((batch_size_n * sequence_length,), 0, dtype=logits.dtype, device=logits.device)
            output_logits[mask] = selected_logits

            output_logits = output_logits.view(batch_size_n, -1).sum(-1)
            out.gpe_score = output_logits.view(int(batch_size_n/self.n_pass), self.n_pass)

        else:
            out.gpe_score = None
            
#         out.rank_loss = None
#         out.kl_d_loss = None


        if pass_label != None:
            pass_label = pass_label[::self.n_pass]
            rank_score = F.log_softmax(out.rank_score/float(config.temp2), dim=-1) # S
            gen_score = F.log_softmax((out.gpe_score/float(config.temp1)), dim=-1) # T
            # K(rs||g) = g.log(rs/g)

            loss_fct1 = nn.KLDivLoss(reduction="batchmean", log_target=True)
            loss_fct3 = nn.KLDivLoss(reduction="batchmean", log_target=True)
            loss_fct2 = nn.CrossEntropyLoss()

            rank_loss = loss_fct2(out.rank_score, pass_label.view(-1))
            kl_d_loss = loss_fct1(rank_score, gen_score.detach()) + loss_fct3(gen_score, rank_score.detach())
            # kl = softmax(gen_score).log(softmax(rank_score)/softmax(gen_score))
            
#             out.rank_loss = rank_loss
#             out.kl_d_loss = kl_d_loss
            loss = rank_loss + config.lamda*kl_d_loss

        ret =  Seq2SeqLMOutput(
            loss=loss,
            logits=out.logits,
            past_key_values=out.past_key_values,
            decoder_hidden_states=out.decoder_hidden_states,
            decoder_attentions=out.decoder_attentions,
            cross_attentions=out.cross_attentions,
            encoder_last_hidden_state=out.encoder_last_hidden_state,
            encoder_hidden_states=out.encoder_hidden_states,
            encoder_attentions=out.encoder_attentions,
        )
        ret.rank_score = out.rank_score
        return ret
    
       
# mod_ckp = "t5-base"
# tokenizer = T5Tokenizer.from_pretrained(mod_ckp)
# question = "question:"
# title = "title:"
# context = "context:"
# eou = "<eou>"
# tokenizer.add_tokens([question, title, context, eou], special_tokens=True)
# config = T5Config.from_pretrained(mod_ckp)
# config.output_hidden_states = True
# config.rank_score_index = tokenizer.convert_tokens_to_ids("<extra_id_80>")
# config.n_pass = 7
# config.temp1 = 1
# config.temp2 = 1
# config.lamda = 0.5
# model = RankT5GPE.from_pretrained(mod_ckp, config=config)

In [14]:
from transformers.modeling_outputs import Seq2SeqLMOutput
class RankT5GPE(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
#         config.rank_score_index = 32019
#         config.n_pass = 7
#         config.output_hidden_states = True
        super().__init__(config)
        self.rank_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.rank_id = config.rank_score_index
        self.n_pass = config.n_pass

    def forward(self, input_ids=None, attention_mask=None, decoder_input_ids=None, labels=None, pass_label=None, **kwargs):

        batch_size_n, seq_len = input_ids.size()
        batch_size = int(batch_size_n/self.n_pass)

        # input_ids = input_ids.view(batch_size*n_pass, -1)
        # attention_mask = attention_mask.view(batch_size*n_pass, -1)
        
        
        
        if labels != None and decoder_input_ids == None:
#             batch_size, decoder_seq_len = labels.size()
#             labels = labels.view(batch_size, 1, decoder_seq_len).contiguous()
#             labels = labels.expand(batch_size, n_pass, decoder_seq_len).contiguous()

#             labels = labels.view(batch_size*n_pass, -1)
            decoder_input_ids = self._shift_right(labels)
#             print(decoder_input_ids.size())


        out = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
        rank_score = self.rank_head(out.decoder_hidden_states[-1][:, 0, :])
#         try:
        out.rank_score = rank_score[:, self.rank_id].view(-1, self.n_pass)
#         except RuntimeError:
#             print(rank_score.size())
#             assert 2 == 3
            

        loss = None

        if labels != None:
            logits = out.logits
            batch_size_n, sequence_length, vocab_size = logits.size()

            logits_flat = logits.view(batch_size_n * sequence_length, vocab_size)
            labels_flat = labels.view(-1)
            mask = (labels_flat != -100)
            arry = torch.arange(batch_size_n * sequence_length).to(logits_flat.device)

            selected_logits = logits_flat[arry[mask], labels_flat[mask]]
            output_logits = torch.full((batch_size_n * sequence_length,), 0, dtype=logits.dtype, device=logits.device)
            output_logits[mask] = selected_logits

            output_logits = output_logits.view(batch_size_n, -1).sum(-1)
            out.gpe_score = output_logits.view(int(batch_size_n/self.n_pass), self.n_pass)

        else:
            out.gpe_score = None
            
#         out.rank_loss = None
#         out.kl_d_loss = None


        if pass_label != None:
            pass_label = pass_label[::self.n_pass]
            rank_score = F.log_softmax(out.rank_score/float(config.temp2), dim=-1) # S
            gen_score = F.softmax((out.gpe_score/float(config.temp1)), dim=-1) # T
            # K(rs||g) = g.log(rs/g)

            loss_fct1 = nn.KLDivLoss(reduction="batchmean")
            loss_fct2 = nn.CrossEntropyLoss()

            rank_loss = loss_fct2(out.rank_score, pass_label.view(-1))
            kl_d_loss = loss_fct1(rank_score, gen_score)
            # kl = softmax(gen_score).log(softmax(rank_score)/softmax(gen_score))
            
#             out.rank_loss = rank_loss
#             out.kl_d_loss = kl_d_loss
            loss = rank_loss + kl_d_loss

        ret =  Seq2SeqLMOutput(
            loss=loss,
            logits=out.logits,
            past_key_values=out.past_key_values,
            decoder_hidden_states=out.decoder_hidden_states,
            decoder_attentions=out.decoder_attentions,
            cross_attentions=out.cross_attentions,
            encoder_last_hidden_state=out.encoder_last_hidden_state,
            encoder_hidden_states=out.encoder_hidden_states,
            encoder_attentions=out.encoder_attentions,
        )
        ret.rank_score = out.rank_score
        return ret

In [16]:
model = RankT5GPE.from_pretrained(model_ckpt)
# model = RankT5.from_pretrained(model_ckpt)

In [17]:
from torchmetrics.retrieval import RetrievalMRR, RetrievalRecall, RetrievalNormalizedDCG
def cal_mrr(rank_score, pass_label):
    pass_label = pass_label.view(-1)[::7]
    target = torch.nn.functional.one_hot(pass_label, 7)
    indexes = torch.arange(rank_score.size()[0]).view(-1, 1).expand(rank_score.size()[0], rank_score.size()[1])
    mr = RetrievalMRR()
    return mr(rank_score, target, indexes=indexes)

def r_at_k(rank_score, pass_label, k):
    pass_label = pass_label.view(-1)[::7]
    target = torch.nn.functional.one_hot(pass_label, 7).to(pass_label.device)
    indexes = torch.arange(rank_score.size()[0]).view(-1, 1).expand(rank_score.size()[0], rank_score.size()[1]).to(pass_label.device)
#     print(rank_score.shape)
    rk = RetrievalRecall(top_k=k)
    return rk(rank_score, target, indexes=indexes)

def ndgc_at_k(rank_score, pass_label, k):
    pass_label = pass_label.view(-1)[::7]
    target = torch.nn.functional.one_hot(pass_label, 7).to(pass_label.device)
    indexes = torch.arange(rank_score.size()[0]).view(-1, 1).expand(rank_score.size()[0], rank_score.size()[1]).to(pass_label.device)
    rk = RetrievalNormalizedDCG(top_k=k)
    return rk(rank_score, target, indexes=indexes)

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer)
dl = DataLoader(test, batch_size=56, collate_fn=data_collator, shuffle=False)


In [20]:
from transformers.modeling_outputs import Seq2SeqLMOutput
from tqdm.notebook import tqdm
model = model.to(device)
pred = None
lab_p = None
batch_len = []
mrr = []
r_k = {}
for i in range(1, 8):
    r_k[i] = []

model.eval()
with torch.no_grad():
    for b in tqdm(dl):
        batch_len.append(int(b["input_ids"].size()[0]))
        input_ids = b["input_ids"].to(device)
        attention_mask = b["attention_mask"].to(device)
        decoder_input_ids = b["decoder_input_ids"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        rank_score = outputs.rank_score
        pass_label = b["pass_label"].to(device)
        
#         print(pass_label.size())
        mrr.append(cal_mrr(rank_score, pass_label))
        if pred == None:
            pred = rank_score
            lab_p = pass_label
        else:
            pred = torch.cat((pred, rank_score), 0)
            lab_p = torch.cat((lab_p, pass_label), 0)

        for i in r_k.keys():
            r_k[i].append(r_at_k(rank_score, pass_label, i))
    
    

  0%|          | 0/460 [00:00<?, ?it/s]

In [21]:
print(f"Mean Reciprocal Rank: {cal_mrr(pred, lab_p)}")

for i in range(1, 8):
    print(f"Recall@{i}: {r_at_k(pred, lab_p, i)}")

for i in range(1, 8):
    print(f"NDGC@{i}: {ndgc_at_k(pred, lab_p, i)}")

Mean Reciprocal Rank: 0.8529314398765564
Recall@1: 0.7606093287467957
Recall@2: 0.8846572041511536
Recall@3: 0.9330794215202332
Recall@4: 0.962459146976471
Recall@5: 0.9834058284759521
Recall@6: 0.9940152168273926
Recall@7: 1.0
NDGC@1: 0.7606093287467957
NDGC@2: 0.838874876499176
NDGC@3: 0.8630859851837158
NDGC@4: 0.8757391571998596
NDGC@5: 0.883842408657074
NDGC@6: 0.8876215815544128
NDGC@7: 0.8896164894104004


In [ ]:
tokenizer.decode(tokenized_wow[155]["input_ids"])

In [21]:
dt = DatasetDict.load_from_disk("wow_rank_ut_base")

In [22]:
tokenizer.decode(dt["test"][155]["input_ids"])

'question: Ah my favorite kind of beer are the stouts. Like a nice milk or chocolate stout with a much larger alcohol content. title: Beer passage: Beer is one of the oldest and most widely consumed alcoholic drinks in the world, and the third most popular drink overall after water and tea. Beer is brewed from cereal grains—most commonly from malted barley, though wheat, maize (corn), and rice are also used. During the brewing process, fermentation of the starch sugars in the wort produces ethanol and carbonation in the resulting beer. Most modern beer is brewed with hops, which add bitterness and other flavours and act as a natural preservative and stabilizing agent. Other flavouring agents such as gruit, herbs, or fruits may be included or used instead of hops. In commercial brewing, the natural carbonation effect is often removed during processing and replaced with forced carbonation. Some of humanity\'s earliest known writings refer to the production and distribution of beer: the C

In [ ]:
sbl = sum(batch_len)/7
batch_len = [a/7 for a in batch_len]
# mrr_score =  

In [ ]:
mrr = [(a*b)/sbl for a, b in zip(mrr, batch_len)]
print(f"Mean Reciprocal Rank: {sum(mrr)}")

In [ ]:
for k in r_k.keys():
    print(f"Recall@{k}: {sum([(a*b)/sbl for a, b in zip(r_k[k], batch_len)])}")

In [ ]:
sum(r_k[3])/len(r_k[7])

In [ ]:
t = torch.randint(1, 34, (20, 5))
p = torch.randint(1, 34, (20, 5))

In [ ]:
t, p

In [ ]:
torch.cat((t, p), 0)

In [ ]:
t

In [ ]:
indexes = torch.arange(x.rank_score.size()[0]).view(-1, 1).expand(x.rank_score.size()[0], x.rank_score.size()[1])

In [ ]:
indexes

In [ ]:
x.rank_score

In [ ]:
from torchmetrics.retrieval import RetrievalMRR
mr = RetrievalMRR()
met = mr(x.rank_score, t, indexes=indexes)

In [ ]:
met

In [ ]:
2428*7

In [ ]:
1 in torch.tensor([1, 4, 5, 2, 0, 3, 6][])

In [ ]:
with torch.no_grad():
    print(x.logits.view(7, -1)[:, 23])

In [ ]:
x.decoder_hidden_states[-1][:, 0, :].size()

In [ ]:
# torch.zeros((70, 1), dtype=int)

In [ ]:
import torch

# Assuming you have a list of rank tensors (output from previous code)
rank_tensors = [
    torch.tensor([3, 1, 4, 2]),
    torch.tensor([2, 3, 1, 4]),
    # Add more rank tensors here
]

# Calculate MRR
mrr_sum = 0.0
total_queries = len(rank_tensors)

for ranks in rank_tensors:
    reciprocal_ranks = 1.0 / ranks.float()
    mrr_sum += reciprocal_ranks.sum()
    print(mrr_sum)

mrr = mrr_sum / total_queries

print("MRR:", mrr.item())